In [1]:
import nltk
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json

In [2]:
path = 'dataset_tweets_WHO.txt'

#convert the text to json
with open(path) as f:
    tweets_json = json.load(f)
    

In [3]:
#inspecting the tweet. all keys are strings btw
tweets_json['1']

{'created_at': 'Wed Oct 13 08:46:17 +0000 2021',
 'id': 1448208458604584960,
 'id_str': '1448208458604584960',
 'full_text': '#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1',
 'truncated': False,
 'display_text_range': [0, 241],
 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [0, 8]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1448208218463875077,
    'id_str': '1448208218463875077',
    'indices': [242, 265],
    'media_url': 'http://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'url': 'https://t.co/jpUQpnu0V1',
    'display_url': 'pic.twitter.com/jpUQpnu0V1',
    'expanded_url': 'https://twitter.com/WHO/status/1448208458604584960/photo/1',
    'type': 'pho

In [4]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line =  line.lower() ## Transform in lowercase
    line =  line.split() ## Tokenize the text to get a list of terms
    line = [word for word in line if word not in stop_words] ##eliminate the stopwords (HINT: use List Comprehension)
    line = [stemmer.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)

    line = [word.replace('#', '') if word[0] == '#' else word for word in line ]
   
    
    """to discuss, should we remove hashtags? 
    yes because if we don't, then something e.x. #covid and covid will have different term frequencies. A document that only
    contains '#covid' is probably very relevant for covid, but will get a tf of 0 if we are searching using the query 'covid'
    
    """
    ## END CODE
    return line

In [5]:
for key in tweets_json:
    key = str(key)
    tweets_json[key]['full_text'] = build_terms(tweets_json[key]['full_text'])

In [6]:
#inspect the first couple of tweets to check
tweets_json['10']

{'created_at': 'Tue Oct 12 21:01:38 +0000 2021',
 'id': 1448031127999074306,
 'id_str': '1448031127999074306',
 'full_text': ['rt',
  '@drtedros:',
  '1951',
  'henrietta',
  'lack',
  'die',
  'cervicalcanc',
  'age',
  '31,',
  '8',
  'month',
  'diagnosis.',
  'although',
  'life',
  'was…'],
 'truncated': False,
 'display_text_range': [0, 139],
 'entities': {'hashtags': [{'text': 'cervicalcancer', 'indices': [48, 63]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'DrTedros',
    'name': 'Tedros Adhanom Ghebreyesus',
    'id': 189868631,
    'id_str': '189868631',
    'indices': [3, 12]}],
  'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 14499829,
  'id_str': '14499829',
  'name': 'World Health Organization (WHO)',
  'screen_name': 'WHO',
  'lo